In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [2]:
import paper

import sympy as sp
from sympy import Eq,symbols
from vessel_manoeuvring_models.symbols import *
from sympy import ImmutableDenseMatrix
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
p2 = p.copy() 

from vessel_manoeuvring_models.substitute_dynamic_symbols import (
    lambdify,
    run,
    expression_to_python_method,
    only_functions,
)

from vessel_manoeuvring_models.models import fossen
from vessel_manoeuvring_models.models.fossen import *


I_z = sp.Symbol(I_z.name, positive=True)
m = sp.Symbol(m.name, positive=True)
x_G = sp.Symbol(x_G.name, positive=True)   
p.Yvdot = sp.Symbol(p.Yvdot.name, negative=True)
p.Yrdot = sp.Symbol(p.Yrdot.name, negative=True)
p.Nvdot = sp.Symbol(p.Nvdot.name, negative=True)
p.Nrdot = sp.Symbol(p.Nrdot.name, negative=True)



In [3]:
def eq(expression, file_name=None):
    print(paper.save_eq(expression, file_name=file_name))

In [4]:
eq_fossen_original = Eq(M*upsilon1d + C*upsilon + D, + tau + tau_wind + tau_wave)
print(paper.save_eq(eq_fossen_original, file_name='fossen_original'))

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\dot{\mathbf{\upsilon}} \mathbf{M} + \mathbf{C} \mathbf{\upsilon} + \mathbf{D} = \mathbf{\tau_{wave}} + \mathbf{\tau_{wind}} + \mathbf{\tau}


In [5]:
eq(fossen.eq_main, 'fossen_modified')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\dot{\mathbf{\upsilon}} \mathbf{M} + \mathbf{C} \mathbf{\upsilon} = \mathbf{D} + \mathbf{\tau_{wave}} + \mathbf{\tau_{wind}} + \mathbf{\tau}


In [6]:
eq_M = Eq(M, M_RB+M_A)
eq(eq_M,'M')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{M} = \mathbf{M_A} + \mathbf{M_{RB}}


In [7]:
eq_C = Eq(C, C_RB+C_A)
eq(eq_C,'C')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{C} = \mathbf{C_A} + \mathbf{C_{RB}}


In [8]:
eq_M_RB = Eq(M_RB,ImmutableDenseMatrix([
    
    [m,0    ,0],
    [0,m    ,m*x_G],
    [0,m*x_G,I_z],
    
]), evaluate=False)
eq(eq_M_RB,'M_RB')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{M_{RB}} = \left[\begin{matrix}m & 0 & 0\\0 & m & m x_{G}\\0 & m x_{G} & I_{z}\end{matrix}\right]


In [9]:
eq_C_RB = Eq(C_RB,ImmutableDenseMatrix([
    
    [0,-m*r,-m*x_G*r],
    [m*r,0,0],
    [m*x_G*r,0,0],
    
]), evaluate=False)
eq(eq_C_RB,'C_RB')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{C_{RB}} = \left[\begin{matrix}0 & - m r & - m r x_{G}\\m r & 0 & 0\\m r x_{G} & 0 & 0\end{matrix}\right]


In [10]:
eq_M_A = Eq(M_A,-ImmutableDenseMatrix([
    
    [p.Xudot,0,0],
    [0,p.Yvdot,p.Yrdot],
    [0,p.Nvdot,p.Nrdot],
    
]), evaluate=False)
eq(eq_M_A,'M_A') 

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{M_A} = \left[\begin{matrix}- X_{\dot{u}} & 0 & 0\\0 & - Y_{\dot{v}} & - Y_{\dot{r}}\\0 & - N_{\dot{v}} & - N_{\dot{r}}\end{matrix}\right]


In [11]:
eq_C_A = Eq(C_A,ImmutableDenseMatrix([
    
    [0,0,p.Yvdot*v+p.Yrdot*r],
    [0,0,-p.Xudot*u],
    [-p.Yvdot*v-p.Yrdot*r,p.Xudot*u,0],
    
]), evaluate=False)
eq(eq_C_A,'C_A') 

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{C_A} = \left[\begin{matrix}0 & 0 & Y_{\dot{r}} r + Y_{\dot{v}} v\\0 & 0 & - X_{\dot{u}} u\\- Y_{\dot{r}} r - Y_{\dot{v}} v & X_{\dot{u}} u & 0\end{matrix}\right]


In [12]:
eq_upsilon = Eq(upsilon,ImmutableDenseMatrix([u,v,r]), evaluate=False)
eq(eq_upsilon,'upsilon')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{\upsilon} = \left[\begin{matrix}u\\v\\r\end{matrix}\right]


In [13]:
eq_upsilon1d = Eq(upsilon1d,ImmutableDenseMatrix([u1d,v1d,r1d]), evaluate=False)
eq(eq_upsilon1d,'upsilon1d')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\dot{\mathbf{\upsilon}} = \left[\begin{matrix}\dot{u}\\\dot{v}\\\dot{r}\end{matrix}\right]


In [14]:
def subs_lhs_rhs(eq_main, equations):
    for eq in equations:
        eq_main = eq_main.subs(eq.lhs,eq.rhs)

    return eq_main

In [15]:
eq_M_expanded = Eq(eq_M.lhs, subs_lhs_rhs(eq_M.rhs, [eq_M_A, eq_M_RB]), evaluate=False)
eq(eq_M_expanded,'M_expanded')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{M} = \left[\begin{matrix}- X_{\dot{u}} + m & 0 & 0\\0 & - Y_{\dot{v}} + m & - Y_{\dot{r}} + m x_{G}\\0 & - N_{\dot{v}} + m x_{G} & I_{z} - N_{\dot{r}}\end{matrix}\right]


In [16]:
eq_C_expanded = Eq(eq_C.lhs, subs_lhs_rhs(eq_C.rhs, [eq_C_A, eq_C_RB]), evaluate=False)
eq(eq_C_expanded,'C_expanded')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{C} = \left[\begin{matrix}0 & - m r & Y_{\dot{r}} r + Y_{\dot{v}} v - m r x_{G}\\m r & 0 & - X_{\dot{u}} u\\- Y_{\dot{r}} r - Y_{\dot{v}} v + m r x_{G} & X_{\dot{u}} u & 0\end{matrix}\right]


## Calculating the acceleration

In [17]:
eq_newton = Eq(upsilon1d*M, sp.solve(fossen.eq_main,upsilon1d*M)[0])
eq(eq_newton,'newton')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\dot{\mathbf{\upsilon}} \mathbf{M} = - \mathbf{C} \mathbf{\upsilon} + \mathbf{D} + \mathbf{\tau_{wave}} + \mathbf{\tau_{wind}} + \mathbf{\tau}


In [18]:
F = symbols("F")
eq_F = Eq(F,eq_newton.rhs)
eq(eq_F,'newton_F')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

F = - \mathbf{C} \mathbf{\upsilon} + \mathbf{D} + \mathbf{\tau_{wave}} + \mathbf{\tau_{wind}} + \mathbf{\tau}


In [19]:
eq_newton2 = eq_newton.subs(eq_F.rhs, eq_F.lhs)
eq(eq_newton2,'newton2')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\dot{\mathbf{\upsilon}} \mathbf{M} = F


In [20]:
M_inv = eq_M_expanded.rhs.inv()
S = symbols("S")
eq_S = Eq(S,M_inv[1,1].args[0].args[0])
eq(eq_S,'S')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

S = I_{z} Y_{\dot{v}} - I_{z} m - N_{\dot{r}} Y_{\dot{v}} + N_{\dot{r}} m + N_{\dot{v}} Y_{\dot{r}} - N_{\dot{v}} m x_{G} - Y_{\dot{r}} m x_{G} + m^{2} x_{G}^{2}


In [21]:
eq_M_inv = Eq(M**(-1),M_inv.subs(eq_S.rhs,eq_S.lhs), evaluate=False)
eq(eq_M_inv,'M_inv')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\frac{1}{\mathbf{M}} = \left[\begin{matrix}\frac{1}{- X_{\dot{u}} + m} & 0 & 0\\0 & \frac{- I_{z} + N_{\dot{r}}}{S} & \frac{- Y_{\dot{r}} + m x_{G}}{S}\\0 & \frac{- N_{\dot{v}} + m x_{G}}{S} & \frac{Y_{\dot{v}} - m}{S}\end{matrix}\right]


## The force vector

In [22]:
eq_D = Eq(D,fossen.eq_D.rhs.subs([(X_D,X_D_),(Y_D,Y_D_),(N_D,N_D_)]), evaluate=False)
eq(eq_D,'D')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

\mathbf{D} = \left[\begin{matrix}X_{D}\\Y_{D}\\N_{D}\end{matrix}\right]


In [23]:
eq_upsilon.rhs


Matrix([
[u],
[v],
[r]])

In [24]:
eq_F_expanded = Eq(F,eq_F.rhs.subs(C*upsilon, eq_C_expanded.rhs*eq_upsilon.rhs))
eq(eq_F_expanded,'F_expanded')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

F = \mathbf{D} + \mathbf{\tau_{wave}} + \mathbf{\tau_{wind}} + \mathbf{\tau} + \left[\begin{matrix}m r v - r \left(Y_{\dot{r}} r + Y_{\dot{v}} v - m r x_{G}\right)\\X_{\dot{u}} r u - m r u\\- X_{\dot{u}} u v - u \left(- Y_{\dot{r}} r - Y_{\dot{v}} v + m r x_{G}\right)\end{matrix}\right]


In [25]:
eq_F_expanded_no_weather = Eq(F,eq_F_expanded.rhs.subs([(tau_wind,0),(tau_wave,0),(tau,0),(D,eq_D.rhs)]), evaluate=False)
eq(eq_F_expanded_no_weather,'F_expanded_no_weather')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

F = \left[\begin{matrix}X_{D} + m r v - r \left(Y_{\dot{r}} r + Y_{\dot{v}} v - m r x_{G}\right)\\X_{\dot{u}} r u + Y_{D} - m r u\\N_{D} - X_{\dot{u}} u v - u \left(- Y_{\dot{r}} r - Y_{\dot{v}} v + m r x_{G}\right)\end{matrix}\right]


In [26]:
rows = []
for row in eq_F_expanded_no_weather.rhs.args[2]:
    rows.append(row.expand().collect(r*u).collect(u*v).collect(r*v))

eq_F_expanded_no_weather2 = Eq(F,sp.ImmutableDenseMatrix(rows), evaluate=False)
eq(eq_F_expanded_no_weather2,'F_expanded_no_weather2')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

F = \left[\begin{matrix}X_{D} - Y_{\dot{r}} r^{2} + m r^{2} x_{G} + r v \left(- Y_{\dot{v}} + m\right)\\Y_{D} + r u \left(X_{\dot{u}} - m\right)\\N_{D} + r u \left(Y_{\dot{r}} - m x_{G}\right) + u v \left(- X_{\dot{u}} + Y_{\dot{v}}\right)\end{matrix}\right]


In [27]:
eq_F_expanded_no_weather.rhs.args[2]

(X_D + m*r*v - r*(Y_{\dot{r}}*r + Y_{\dot{v}}*v - m*r*x_G), X_{\dot{u}}*r*u + Y_D - m*r*u, N_D - X_{\dot{u}}*u*v - u*(-Y_{\dot{r}}*r - Y_{\dot{v}}*v + m*r*x_G))

In [28]:
eq_F_expanded_no_weather.rhs[0].expand().collect(r*u).collect(u*v).collect(r*v)

X_D - Y_{\dot{r}}*r**2 + m*r**2*x_G + r*v*(-Y_{\dot{v}} + m)

In [29]:
eq_F_expanded_no_weather.rhs[1].expand().collect(r*u).collect(u*v)

Y_D + r*u*(X_{\dot{u}} - m)

In [30]:
eq_F_expanded_no_weather.rhs[2].expand().collect(r*u).collect(u*v)

N_D + r*u*(Y_{\dot{r}} - m*x_G) + u*v*(-X_{\dot{u}} + Y_{\dot{v}})

## $\mathbf{M}$ invertable?

In [31]:
M_ = eq_M_expanded.rhs
M_


Matrix([
[-X_{\dot{u}} + m,                    0,                    0],
[               0,     -Y_{\dot{v}} + m, -Y_{\dot{r}} + m*x_G],
[               0, -N_{\dot{v}} + m*x_G,    I_z - N_{\dot{r}}]])

In [32]:
M_[1:,1:]


Matrix([
[    -Y_{\dot{v}} + m, -Y_{\dot{r}} + m*x_G],
[-N_{\dot{v}} + m*x_G,    I_z - N_{\dot{r}}]])

### square?

In [33]:
assert M_.shape[0] ==  M_.shape[1]

### Full rank?

In [34]:
is_full_rank = M_.rank() == M_.shape[0]
assert is_full_rank

### nonzero determinant?

In [65]:
#det = -M_[1:,1:].det()
det = eq_S.rhs
det

I_z*Y_{\dot{v}} - I_z*m - N_{\dot{r}}*Y_{\dot{v}} + N_{\dot{r}}*m + N_{\dot{v}}*Y_{\dot{r}} - N_{\dot{v}}*m*x_G - Y_{\dot{r}}*m*x_G + m**2*x_G**2

In [66]:
subs_sign = {symbol:sp.Abs(sp.Symbol(symbol.name)) if sp.sign(symbol)==1 else -sp.Abs(sp.Symbol(symbol.name)) for symbol in det.free_symbols}
det_sign = sp.simplify(det.subs(subs_sign))
det_sign

-Abs(I_z*Y_{\dot{v}}) - Abs(I_z*m) - Abs(N_{\dot{r}}*Y_{\dot{v}}) - Abs(N_{\dot{r}}*m) + Abs(N_{\dot{v}}*Y_{\dot{r}}) + Abs(m**2*x_G**2) + Abs(N_{\dot{v}}*m*x_G) + Abs(Y_{\dot{r}}*m*x_G)

In [68]:
eq_S_sign = Eq(S,det_sign)
eq(eq_S_sign,'eq_S_sign')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

S = - \left|{I_{z} Y_{\dot{v}}}\right| - \left|{I_{z} m}\right| - \left|{N_{\dot{r}} Y_{\dot{v}}}\right| - \left|{N_{\dot{r}} m}\right| + \left|{N_{\dot{v}} Y_{\dot{r}}}\right| + \left|{m^{2} x_{G}^{2}}\right| + \left|{N_{\dot{v}} m x_{G}}\right| + \left|{Y_{\dot{r}} m x_{G}}\right|


In [37]:
sub_python2 = {value:key for key,value in p2.items()}

In [43]:
ship_name = "optiwise"
model = catalog.load(f"{ship_name}.models_VCT_MMG_rudder_MDL")["quadratic"]()

[02/19/25 13:24:06] WARNING  /home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/pa ]8;id=712782;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=271587;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             rtitioned_dataset.py:200: KedroDeprecationWarning:                                    
                             'PartitionedDataset' has been moved to `kedro-datasets` and will be                   
                             removed in Kedro 0.19.0.                                                              
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/__ ]8;id=10084;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=216141;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             init__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been                     
                             renamed to 'AbstractDataset', and the alias will be removed in Kedro                  
                             0.19.0                                                                                
                               return getattr(kedro.io.core, name)                                                 
                                                                                                                   

[02/19/25 13:24:07] INFO     Loading data from 'optiwise.models_VCT_MMG_rudder_MDL'             ]8;id=227756;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=577769;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (PartitionedDataset)...                                                               

In [51]:
mask = pd.Series(model.parameters).index.str.contains('dot')
parameters = pd.concat((
    pd.Series(model.ship_parameters_prime)[['I_z','m','x_G']],
    pd.Series(model.parameters)[mask],
))

In [52]:
calculations = {}

for part in det_sign.args:
    calculations[part] = run(lambdify(part.subs(sub_python2)),parameters) 

calculations = pd.Series(calculations)

In [53]:
calculations


-Abs(I_z*Y_{\dot{v}})           -1.779705e-05
-Abs(I_z*m)                     -2.071602e-05
-Abs(N_{\dot{r}}*Y_{\dot{v}})   -1.124506e-05
-Abs(N_{\dot{r}}*m)             -1.308941e-05
Abs(N_{\dot{v}}*Y_{\dot{r}})     2.883690e-07
Abs(m**2*x_G**2)                 4.417278e-07
Abs(N_{\dot{v}}*m*x_G)           3.569042e-07
Abs(Y_{\dot{r}}*m*x_G)           3.569042e-07
dtype: float64

In [61]:
df_calulations = pd.DataFrame()
df_calulations['value'] = calculations
df_calulations['part'] =  pd.Series(df_calulations.index, index=df_calulations.index).apply(lambda x:f"${sp.latex(x)}$")
df_calulations['magnitude'] = df_calulations['value'].apply(lambda x: np.sign(x)*10**np.floor(np.log10(abs(x))))

In [62]:
df_calulations[['part','value','magnitude']]

,part,value,magnitude
-Abs(I_z*Y_{\dot{v}}),$- \left|{I_{z} Y_{\dot{v}}}\right|$,-1.779705e-05,-1.000000e-05
-Abs(I_z*m),$- \left|{I_{z} m}\right|$,-2.071602e-05,-1.000000e-05
-Abs(N_{\dot{r}}*Y_{\dot{v}}),$- \left|{N_{\dot{r}} Y_{\dot{v}}}\right|$,-1.124506e-05,-1.000000e-05
-Abs(N_{\dot{r}}*m),$- \left|{N_{\dot{r}} m}\right|$,-1.308941e-05,-1.000000e-05
Abs(N_{\dot{v}}*Y_{\dot{r}}),$\left|{N_{\dot{v}} Y_{\dot{r}}}\right|$,2.883690e-07,1.000000e-07
Abs(m**2*x_G**2),$\left|{m^{2} x_{G}^{2}}\right|$,4.417278e-07,1.000000e-07
Abs(N_{\dot{v}}*m*x_G),$\left|{N_{\dot{v}} m x_{G}}\right|$,3.569042e-07,1.000000e-07
Abs(Y_{\dot{r}}*m*x_G),$\left|{Y_{\dot{r}} m x_{G}}\right|$,3.569042e-07,1.000000e-07


In [63]:
df_calulations[['part','magnitude']].to_csv(paper.file_path_with_nb_ref("determinant.csv", directory='tables'), index=False)

In [60]:
df_calulations['value'].apply(lambda x: np.sign(x)*10**np.floor(np.log10(abs(x))))


-Abs(I_z*Y_{\dot{v}})           -1.000000e-05
-Abs(I_z*m)                     -1.000000e-05
-Abs(N_{\dot{r}}*Y_{\dot{v}})   -1.000000e-05
-Abs(N_{\dot{r}}*m)             -1.000000e-05
Abs(N_{\dot{v}}*Y_{\dot{r}})     1.000000e-07
Abs(m**2*x_G**2)                 1.000000e-07
Abs(N_{\dot{v}}*m*x_G)           1.000000e-07
Abs(Y_{\dot{r}}*m*x_G)           1.000000e-07
Name: value, dtype: float64

In [ ]:
eq_det = Eq(det,0)
eq_det = Eq(eq_det.lhs+I_z*m,eq_det.rhs+I_z*m)
eq_det = Eq(eq_det.lhs+p.Nrdot*p.Yvdot,eq_det.rhs+p.Nrdot*p.Yvdot)
eq_det = Eq(eq_det.lhs+p.Nvdot*m*x_G,eq_det.rhs+p.Nvdot*m*x_G)
eq_det = Eq(eq_det.lhs+p.Yrdot*m*x_G,eq_det.rhs+p.Yrdot*m*x_G)


In [ ]:
eq_det

In [ ]:
det.args

In [ ]:
parts_sign = []

for arg in det.args:
    if sp.sign(arg) == 1:
        parts_sign.append(sp.Abs(arg))
    else:
        parts_sign.append(-sp.Abs(arg))

In [ ]:
sp.Abs(sp.Symbol(p.Yvdot.name))

In [ ]:
sp.simplify(sp.Abs(sp.symbols('l'))*sp.Abs(sp.symbols('k')))

In [ ]:
sub_python = {value:key for key,value in p.items()}

In [ ]:
calculations_lhs = {}

for part in eq_det.lhs.args:
    calculations_lhs[part] = run(lambdify(part.subs(sub_python)),parameters) 

calculations_lhs = pd.Series(calculations_lhs)

In [ ]:
calculations_rhs = {}

for part in eq_det.rhs.args:
    calculations_rhs[part] = run(lambdify(part.subs(sub_python)),parameters) 

calculations_rhs = pd.Series(calculations_rhs)

In [ ]:
df_calulations_lhs = pd.DataFrame()
df_calulations_lhs['value'] = calculations_lhs
df_calulations_lhs['side'] = 'LHS'

df_calulations_rhs = pd.DataFrame()
df_calulations_rhs['value'] = calculations_rhs
df_calulations_rhs['side'] = 'RHS'


In [ ]:
df_calulations = pd.concat((df_calulations_lhs,df_calulations_rhs),axis=0)
df_calulations['abs(value)'] = df_calulations['value'].abs()
df_calulations['sign'] = np.sign(df_calulations['value'])
df_calulations['sign'] = df_calulations['sign'].astype(int)
df_calulations.sort_values(by='abs(value)', ascending=False, inplace=True)
df_calulations['exponent'] = df_calulations['value'].apply(lambda x: int(np.floor(np.log10(abs(x)))))
df_calulations['part'] =  pd.Series(df_calulations.index, index=df_calulations.index).apply(lambda x:f"${sp.latex(x)}$")

In [ ]:
df_calulations

In [ ]:
df_calulations[['part','sign','exponent']]

In [ ]:
df_calulations[['part','sign','exponent']].to_csv(paper.file_path_with_nb_ref("determinant.csv", directory='tables'), index=False)

In [ ]:
M_.inv()

In [ ]:
-M_[1:,1:].det()